## Version Description
In this version, we deploy a Hybrid PINN to predict only one correction factor for the MTR ( MTR-600 ) by the neural network. We implement line search ([Secant Method](https://nickcdryan.com/2017/09/13/root-finding-algorithms-in-python-line-search-bisection-secant-newton-raphson-boydens-inverse-quadratic-interpolation-brents/)) to adaptively find the learning rate.
* SingleHoV is looped till it attains convergence, so `batch size = 1` is used.
* Two extra features `x1` and `x2` are engineered and the input features used are `['x1','x2']` only. `TZ6_flow` is used as a seperate input not part of `X` (set of all features in train dataset).
* **Linear** (Identity) Activation function is used in the **hidden and output layers**.

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# import required libraries
import random
import pandas as pd
import numpy as np

from functools import reduce
import sys

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file,output_notebook
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [3]:
np.random.seed(42)

# import LTR and corresponding Restrictor dataframe
Restrictor_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_RestrictorData_LTR.csv')
LTR_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_LTRData.csv')
obs_HOVs = pd.read_csv('../data/LTR_dataset.csv', usecols = ['HoV'])[['HoV']]
LTR_data[['AMBP','MIXP']] *= 100 # Convert hPa to Pa
zero_MIXT_hoV_idx = LTR_data.index[LTR_data['MIXT'] == 20].tolist() # Points where originally, MIXT == 0 and substituted with 20° C
LTR_data[['MIXT','AMBT']] += 273.15 # Convert Celcius to Kelvin
LTR_data[['TZ6_Flow']] /= 1000 # Convert l/s to cu.m/s
print('Total LTR points:',len(LTR_data))

## Calculate Density = (0.5* Pmix + Pamb) / (R * Tmix) where R = 287 is the Gas constant in Ideal Gas Law
idx = LTR_data.columns.get_loc("AMBT")
LTR_data.insert(loc=(idx+1), column='Density(kg/m³)', value=np.round((0.5 * LTR_data['MIXP'] + LTR_data['AMBP'] ) / (LTR_data['MIXT']*287),4))
target_df = reduce(lambda left,right: pd.merge(left,right,left_on='HoV',right_on='HoV'), [LTR_data,Restrictor_data])
target_df['HoV'] = obs_HOVs.values
target_df.head()

Total LTR points: 34


,HoV,CAOLH_C66-C68,CAOLH_C68-C70,CAOLH_C70-C72,CAOLH_C72-C74,CAOLH_C74-C76,CAOLH_C76-C78,CAORH_C66-C68,CAORH_C68-C70,CAORH_C70-C72,...,R632_HS1,R633_HS1,R634_HS1,R635_HS1,R636_HS1,R637_HS1,R638_HS1,R639_HS1,R640_HS1,R641_HS1
0,A1,36.071000,38.802000,36.796000,36.794000,39.100000,38.318000,37.055000,39.596000,38.333000,...,41,41,41,41,41,41,41,41,41,41
1,A2,36.061000,38.697000,37.156000,36.738000,38.957000,38.326000,36.501000,40.797000,39.284000,...,41,41,41,41,41,41,41,41,41,41
2,A3,37.504234,40.911735,38.412733,37.859887,36.585486,38.504564,36.161971,37.068225,37.893405,...,41,41,41,41,41,41,41,41,41,41
3,A4,36.383413,39.101836,38.110486,37.597728,36.190671,37.706898,36.668551,37.759174,38.497755,...,41,41,41,41,41,41,41,41,41,41
4,A5,36.897000,39.925000,38.704000,38.175000,40.591000,39.913000,37.641000,38.309000,38.600000,...,41,41,41,41,41,41,41,41,41,41


In [4]:
V_max_org = target_df['TZ6_Flow'].max()
print('Maximum TZ6 Flow:',V_max_org)

Maximum TZ6 Flow: 0.7640123719999998


In [5]:
MIXP = [1800, 2600, 3400]
# Define Restrictor Labels
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']
CAORS = ['R620_HS1','R620_HS2','R620_HS3','R621_HS1','R621_HS2','R621_HS3']
MHRS = CLRS + CAORS
LAORS = ['R630_HS1','R631_HS1','R632_HS1','R633_HS1','R634_HS1','R635_HS1','R636_HS1','R637_HS1','R638_HS1','R639_HS1','R640_HS1','R641_HS1']

In [6]:
# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461,
                 'R620_HS1':0.01090,'R620_HS2':0.00470,'R620_HS3':0.01085,'R621_HS1':0.01085,'R621_HS2':0.00470,'R621_HS3':0.01090}

## Define Initial Correction factor values from CFD
# Correction Factor for Duct Part `TZ6_zmix:
TZ6_zmix_cf = 1 # Start with initial c_f =1 

#3D Correction Factor for MTR
MTR_cf = 1.1664876228437


#3D Correction Factors for MultiHole Restrictors
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569,
          'R620_HS1':0.757121843184036,'R620_HS2':0.773919779037984,'R620_HS3':0.789028593182055,
          'R621_HS1':0.714811824527744,'R621_HS2':0.866271631648556,'R621_HS3':0.822447592395462}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [7]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [8]:
target_df[MTR+'_Area'],target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Area','R600_HD_Zeta3D']].head()

,R600_HD,R600_HD_Area,R600_HD_Zeta3D
0,148,0.017203,4.949375
1,149,0.017437,4.730821
2,152,0.018146,4.127180
3,154,0.018627,3.764504
4,148,0.017203,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [9]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [10]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    target_df[clr+'_Area'], target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORS:
    CAOR_nr_holes = target_df[caor].values
    target_df[caor+'_Area'],target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,8,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

**Store Loss Coefficient ($\zeta$) Values and Names for all Restrictors:**

In [11]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
area_col_names = [col for col in target_df.columns if 'Area' in col]
area_col_names_idx = [target_df.columns.get_loc(col) for col in area_col_names if col in target_df]
pt_var_idx = [target_df.columns.get_loc(col) for col in ['MIXP','AMBP','AMBT'] if col in target_df]
req_flow_rates_idx = [target_df.columns.get_loc(col) for col in ['TZ6_Flow'] if col in target_df]
# print(zeta_col_names_idx)
print(zeta_col_names)

['R600_HD_Zeta3D', 'R610_HS1_Zeta3D', 'R611_HS1_Zeta3D', 'R612_HS1_Zeta3D', 'R613_HS1_Zeta3D', 'R620_HS1_Zeta3D', 'R620_HS2_Zeta3D', 'R620_HS3_Zeta3D', 'R621_HS1_Zeta3D', 'R621_HS2_Zeta3D', 'R621_HS3_Zeta3D']


In [12]:
LTR_zeta_df_col_idx = sorted([0] + req_flow_rates_idx + pt_var_idx + zeta_col_names_idx)
target_df.iloc[:,LTR_zeta_df_col_idx].to_csv('data_output/LTR_Zeta_df.csv', index=False)

In [13]:
LTR_zeta_df = target_df[['HoV','MIXP','AMBP','AMBT','TZ6_Flow']+zeta_col_names]
final_df = target_df[['HoV','MIXP','Density(kg/m³)','R600_HD','R600_HD_Area','R600_HD_Zeta3D','TZ6_Flow']]
final_df.reset_index(drop=True,inplace = True)
final_df.head()

,HoV,MIXP,Density(kg/m³),R600_HD,R600_HD_Area,R600_HD_Zeta3D,TZ6_Flow
0,A1,2600.0000,1.2207,148,0.017203,4.949375,0.738083
1,A2,2600.0000,1.2228,149,0.017437,4.730821,0.737775
2,A3,2606.1928,1.2090,152,0.018146,4.127180,0.740980
3,A4,2599.8998,1.2368,154,0.018627,3.764504,0.732112
4,A5,2600.0000,1.2380,148,0.017203,4.949375,0.741828


### Feature Engineering

In [14]:
final_df['xf1'] = ( final_df['MIXP']/final_df['R600_HD_Zeta3D'] ) * (2/final_df['Density(kg/m³)']) * (MTR_DuctArea/ final_df['TZ6_Flow'])**2
final_df['xf2'] = ( 1 /final_df['R600_HD_Zeta3D'] )
final_df.drop(columns = ['MIXP','Density(kg/m³)','R600_HD','R600_HD_Zeta3D','R600_HD_Area','TZ6_Flow'], inplace = True)

### Train Data Preperation

In [15]:
input_features = ['xf1','xf2']
## Using un-scaled values
train_data = final_df[input_features]
print("Number of training samples:", len(train_data))
train_data

Number of training samples: 34


,xf1,xf2
0,1.895786,0.202046
1,1.981615,0.211380
2,2.282967,0.242296
3,2.500234,0.265639
4,1.850468,0.202046
5,1.869475,0.202046
6,1.899464,0.202046
7,2.398352,0.253672
8,2.319517,0.242296
9,2.044012,0.221181


In [16]:
# Store indices of train, test and validation data
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = target_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network

In [17]:
# Define custom MSE function
MSE = lambda y_hat,y_true: np.mean((np.asarray(y_hat)-np.asarray(y_true))**2)

**Train the Neural Network**

In [18]:
### Set the hyperparameters here ###
iterations = 100
lr0, lr1 = 0, 1
hidden_nodes = 1 ### Important Parameter
output_nodes = 1
batch_sample_size = 2
MTR_epsi = np.float64(MTR_cf * 0.01)

In [19]:
from Hybrid_PINN_v2_1 import PINN

In [20]:
%%time
N_i = X_train.shape[1] # no. of features
network = PINN(N_i, hidden_nodes, output_nodes, MTR_epsi)

MSE_flowloss_hist = []
err_loss_hist = []
losses_hist = {'HoV':[],'MSE_flowloss':[],'Error_loss':[]}
c_f_hist = {'HoV':[],'c_f_hat':[],'FlowRate_Diff_(LTR-FDDN)':[],'f1':[],'f2':[]}

# Set Initial Weights and Gradient Updates
w_ih, w_ho, dw_i_h, dw_h_o = None, None, None, None

for idx in train_data_idx:    
     # Go through one record at a time from the training data set        
    print('Old_Zeta:',target_df[MTR+'_Zeta3D'].loc[idx])    
    X_train = train_data[input_features].loc[[idx]].values
    dia_train = np.atleast_1d(train_R600HD_series.loc[idx])
    HoV = target_df[['HoV']].loc[idx].values.tolist()
    for ii in range(iterations):                
        ## Initialize with default weights    
        w0_i_h, w0_h_o = network.calc_new_weights(lr0, delta_weights_i_h = dw_i_h, delta_weights_h_o = dw_h_o, w_ih = w_ih, w_ho = w_ho)
        print('\nTRAINING ITERATION {} - w_0:'.format(ii), list(w0_i_h) + list(w0_h_o))                                                                        
        V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, hidden_outputs, _ = network.train(X_train, dia_train,[idx], w0_i_h, w0_h_o, MTR_DuctArea, target_df, zeta_col_names)
        _, dE_dw_ih0, dE_dw_ho0 = network.gradient_update(V_LTR_true, V_hat_FDDN , V_hat_epsi_FDDN, cf_array, X_train, hidden_outputs, dw_i_h, dw_h_o, w0_h_o, V_max_org)    
        f0 = network.cust_f_alpha(dE_dw_ih0, dE_dw_ho0, dE_dw_ih0, dE_dw_ho0 )        
        print('f0:', f0)
        w1_i_h, w1_h_o = network.calc_new_weights(lr1, dE_dw_ih0, dE_dw_ho0, w_ih = w0_i_h, w_ho = w0_h_o)
        V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, hidden_outputs, _ = network.train(X_train, dia_train, [idx], w1_i_h, w1_h_o, MTR_DuctArea, target_df, zeta_col_names)
        _,dE_dw_ih1, dE_dw_ho1 = network.gradient_update(V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, X_train, hidden_outputs, dw_i_h, dw_h_o, w1_h_o, V_max_org )
        f1 = network.cust_f_alpha(dE_dw_ih0, dE_dw_ho0, dE_dw_ih1, dE_dw_ho1)        
        print('f1:', f1)
        lr2 = network.calc_lr2( f0, f1, lr0, lr1)    
        w2_i_h, w2_h_o = network.calc_new_weights(lr2, dE_dw_ih0, dE_dw_ho0, w_ih = w0_i_h, w_ho = w0_h_o)    
        V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, hidden_outputs, break_flag_arr = network.train(X_train, dia_train, [idx], w2_i_h, w2_h_o, MTR_DuctArea, target_df, zeta_col_names)
        error, dE_dw_ih2, dE_dw_ho2 = network.gradient_update(V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, X_train, hidden_outputs, dw_i_h, dw_h_o, w2_h_o, V_max_org)
        f2 = network.cust_f_alpha(dE_dw_ih0, dE_dw_ho0, dE_dw_ih2, dE_dw_ho2)        
        print('f2:', f2)
        w_ih, w_ho = w2_i_h, w2_h_o
        dw_i_h, dw_h_o = dE_dw_ih2, dE_dw_ho2

        # Print out the training progress        
        MSE_flowloss = MSE(V_hat_FDDN, V_LTR_true) # MSE between FDDN predicted flow rate and true flow rate on UN-scaled original values in cu.m/s
        sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations))+ "% ... MSE FLowloss: " + str(round(MSE_flowloss,4))[:5] + " ... Error Function loss: " + str(round(error,4))) # For truncated version of the error
        print('... Training Iteration:',ii)    
        sys.stdout.flush()
        MSE_flowloss_hist.append(MSE_flowloss)
        err_loss_hist.append(error) # For truncated version of the error            
        if (1 in break_flag_arr) or (ii == (iterations - 1)): # To remove converged HoV
            print('EARLY STOPPING ACTIVATED - Terminating Neural Network Training')
            print('MSE FlowLoss History for {}:'.format(HoV[0]),MSE_flowloss_hist)
            print('Error Loss History for {}:'.format(HoV[0]),err_loss_hist)
            losses_hist['HoV'].append(HoV[0])            
            losses_hist['MSE_flowloss'].append(MSE_flowloss_hist)
            losses_hist['Error_loss'].append(err_loss_hist)
            c_f_hist['HoV'].append(HoV[0])
            c_f_hist['FlowRate_Diff_(LTR-FDDN)'].append( V_LTR_true[0][0] - V_hat_FDDN[0][0])
            c_f_hist['c_f_hat'].append(cf_array[0])
            c_f_hist['f1'].append(f1)
            c_f_hist['f2'].append(f2)
            err_loss_hist = []
            MSE_flowloss_hist = []
            break

Initial Weights_i_h: [array([1]), array([0.1])]
Initial Weights_h_o: [array([1])]
Old_Zeta: 4.949375283472444

CALC NEW WEIGHTS - Learning Rate Used: 0
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([0]), array([0]), array([0])]
CALC NEW WEIGHTS - new_weights_i_h: [array([1]), array([0.1])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

TRAINING ITERATION 0 - w_0: [array([1]), array([0.1]), array([1])]

NN output (C_f): 1.9159908486948936
New_Zeta: 8.129497102397062
FDDN Solver Output: [[64.2 30.7 96.9 63.4 33.5 103 10.1 21.9 21.9 21.9 21.4 21.2 9.9 21.6 21.7 21.4 21.1 21.1]]
New_Zeta_epsi: 8.178990855231786
FDDN Solver Output: [[64.1 30.6 96.7 63.2 33.4 103 10.1 21.9 21.8 21.9 21.4 21.1 9.89 21.5 21.7 21.3 21.1 21.1]]
Row ID: 0
HoV: ['A1']
Flow Rate Difference (LTR - FDDN): 0.11101802247714976 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([0.0356]), array([0.0038])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 0.0012835642461

FDDN Solver Output: [[75.6 36.1 114 74.6 39.4 122 11.9 25.8 25.7 25.8 25.2 24.9 11.7 25.4 25.6 25.2 24.8 24.9]]
New_Zeta_epsi: 4.3955981729124245
FDDN Solver Output: [[75.4 36 114 74.4 39.3 121 11.9 25.7 25.7 25.7 25.2 24.8 11.6 25.3 25.5 25.1 24.8 24.8]]
Row ID: 0
HoV: ['A1']
Flow Rate Difference (LTR - FDDN): 7.179836316406618e-05 cu.m/s
% Diff between flowrates 0.009728154623796939 %
GRADIENT_UPDATE - dw_ih: [array([3.69e-05]), array([3.93e-06])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f2: -2.028589700587739e-07
Progress: 2.0% ... MSE FLowloss: 0.0 ... Error Function loss: 0.0... Training Iteration: 2
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSE FlowLoss History for A1: [0.0009764357416136609, 0.00010340642799070019, 5.155004953039136e-09]
Error Loss History for A1: [0.0008363978150725182, 8.857614152155417e-05, 4.415687275318825e-09]
Old_Zeta: 4.7308206417914045

CALC NEW WEIGHTS - Learning Rate Used: 

FDDN Solver Output: [[75.5 36 114 74.5 39.3 121 11.9 25.7 25.7 25.7 25.2 24.8 11.6 25.3 25.5 25.1 24.8 24.8]]
New_Zeta_epsi: 4.106144767157044
FDDN Solver Output: [[75.4 35.9 114 74.3 39.3 121 11.9 25.7 25.6 25.7 25.1 24.8 11.6 25.3 25.5 25.1 24.8 24.8]]
Row ID: 3
HoV: ['A4']
Flow Rate Difference (LTR - FDDN): -0.004585961451015197 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00242]), array([-0.000257])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 5.92695189660272e-06

CALC NEW WEIGHTS - Learning Rate Used: 1
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00242]), array([-0.000257]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.499]), array([0.0466])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.502]), array([0.0469])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

NN output (C_f): 1.26680654053849
New_Zeta:

FDDN Solver Output: [[76.4 36.4 115 75.3 39.8 123 12.1 26 26 26 25.5 25.1 11.8 25.6 25.8 25.4 25.1 25.1]]
New_Zeta_epsi: 4.009919862767694
FDDN Solver Output: [[76.2 36.3 115 75.1 39.7 122 12 25.9 25.9 26 25.4 25.1 11.7 25.5 25.8 25.3 25 25]]
Row ID: 5
HoV: ['A6']
Flow Rate Difference (LTR - FDDN): -0.0018149228720238941 cu.m/s
% Diff between flowrates 0.24379691406387843 %
GRADIENT_UPDATE - dw_ih: [array([-0.000979]), array([-0.000106])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f2: -5.3431777482010154e-06
Progress: 0.0% ... MSE FLowloss: 0.0 ... Error Function loss: 0.0... Training Iteration: 0
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSE FlowLoss History for A6: [3.2939450313954605e-06]
Error Loss History for A6: [2.8215358264898605e-06]
Old_Zeta: 4.949375283472444

CALC NEW WEIGHTS - Learning Rate Used: 0
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.000979]), array([-0.000106]), array([0])]
C

FDDN Solver Output: [[77 36.7 116 74.6 39.4 122 11.9 25.8 25.7 25.8 25.2 24.9 11.7 25.4 25.6 25.2 24.9 24.9]]
New_Zeta_epsi: 4.358477155988943
FDDN Solver Output: [[76.8 36.7 116 74.5 39.3 121 11.9 25.7 25.7 25.7 25.2 24.8 11.6 25.3 25.5 25.1 24.8 24.8]]
Row ID: 8
HoV: ['C3']
Flow Rate Difference (LTR - FDDN): -0.002661390601208846 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00142]), array([-0.000149])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 2.0449589142908696e-06

CALC NEW WEIGHTS - Learning Rate Used: 1
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00142]), array([-0.000149]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.521]), array([0.0489])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.522]), array([0.0491])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

NN output (C_f): 1.2235304052379805
New_Z

...

EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSE FlowLoss History for M1: [2.937920779334254e-05, 6.769963160443745e-08]
Error Loss History for M1: [2.5165716656688814e-05, 5.79903229080791e-08]
Old_Zeta: 3.7645035449506787

CALC NEW WEIGHTS - Learning Rate Used: 0
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00014]), array([-1.55e-05]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.477]), array([0.0417])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.477]), array([0.0417])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

TRAINING ITERATION 0 - w_0: [array([0.477]), array([0.0417]), array([1])]

NN output (C_f): 1.1672216136728726
New_Zeta: 3.7668722893970585
FDDN Solver Output: [[79.6 38 120 77.2 40.8 126 12.6 27.1 27.1 27.1 26.6 26.2 12.2 26.5 26.7 26.3 25.9 26]]
New_Zeta_epsi: 3.8045173248465654
FDDN Solver Output: [[79.5 37.9 120 77 40.7 126 1

FDDN Solver Output: [[74.6 35.6 112 73.2 38.7 119 11.8 25.6 25.5 25.6 25 24.7 11.6 25.2 25.4 25 24.6 24.7]]
New_Zeta_epsi: 4.212526937970563
FDDN Solver Output: [[74.4 35.5 112 73 38.6 119 11.8 25.5 25.5 25.5 25 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
Row ID: 25
HoV: ['Q1']
Flow Rate Difference (LTR - FDDN): -0.003236146646017568 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00167]), array([-0.000171])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f1: 3.347861167797941e-06
Learning Rate 2: 6.164020906381514

CALC NEW WEIGHTS - Learning Rate Used: 6.164020906381514
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00199]), array([-0.000205]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.493]), array([0.0434])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.505]), array([0.0447])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1]

FDDN Solver Output: [[76.6 36.6 116 73.6 38.9 120 12.1 26.1 26 26.1 25.6 25.2 11.8 25.7 25.9 25.5 25.2 25.2]]
Row ID: 27
HoV: ['S1']
Flow Rate Difference (LTR - FDDN): -0.00096260517178548 cu.m/s
% Diff between flowrates 0.12963391221975057 %
GRADIENT_UPDATE - dw_ih: [array([-0.00051]), array([-5.44e-05])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f2: -1.7623992280329718e-06
Progress: 0.0% ... MSE FLowloss: 0.0 ... Error Function loss: 0.0... Training Iteration: 0
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
MSE FlowLoss History for S1: [9.266087167481534e-07]
Error Loss History for S1: [7.937168551762715e-07]
Old_Zeta: 4.949375283472444

CALC NEW WEIGHTS - Learning Rate Used: 0
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00051]), array([-5.44e-05]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.495]), array([0.0435])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o:

FDDN Solver Output: [[76.7 36.6 116 74.3 39.3 121 11.9 25.7 25.6 25.7 25.1 24.8 11.6 25.3 25.5 25.1 24.8 24.8]]
New_Zeta_epsi: 4.139365062078327
FDDN Solver Output: [[76.5 36.5 115 74.2 39.2 121 11.9 25.6 25.6 25.6 25.1 24.7 11.6 25.2 25.4 25 24.7 24.7]]
Row ID: 30
HoV: ['S4']
Flow Rate Difference (LTR - FDDN): -0.005074585350561822 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00271]), array([-0.000287])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 7.445782089689984e-06

CALC NEW WEIGHTS - Learning Rate Used: 1
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00271]), array([-0.000287]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.5]), array([0.0441])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.503]), array([0.0444])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

NN output (C_f): 1.0620508290859585
New_Zet

FDDN Solver Output: [[74.5 35.5 112 73.5 38.8 120 11.8 25.4 25.3 25.4 24.8 24.5 11.5 25 25.2 24.8 24.5 24.5]]
New_Zeta_epsi: 4.7498638147734535
FDDN Solver Output: [[74.3 35.5 112 73.3 38.7 119 11.7 25.3 25.3 25.3 24.8 24.4 11.5 24.9 25.1 24.7 24.4 24.4]]
Row ID: 32
HoV: ['T1']
Flow Rate Difference (LTR - FDDN): -0.004365194550516538 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00223]), array([-0.000225])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f2: 3.178320334893947e-05
Progress: 0.0% ... MSE FLowloss: 0.0 ... Error Function loss: 0.0... Training Iteration: 0

CALC NEW WEIGHTS - Learning Rate Used: 0
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00223]), array([-0.000225]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.548]), array([0.0486])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.548]), array([0.0486])]

In [21]:
cf_df = pd.DataFrame(data=c_f_hist)
losses = pd.DataFrame(data=losses_hist)

In [22]:
mseflow_last_loss = []
error_last_loss = []
for i in range(len(losses)):
    loss1 = losses.MSE_flowloss[i][-1]
    loss2 = losses.Error_loss[i][-1]
    mseflow_last_loss.append(loss1)
    error_last_loss.append(loss2)

cf_df['Final_MSELoss'] = mseflow_last_loss
cf_df['Final_ErrorLoss'] = error_last_loss

In [23]:
cf_df['MaxEpochs'] = None # Initialize Empty Column for storing epochs trained

for i in range(len(losses['HoV'])):
    nr_epochs_trained = len(losses['MSE_flowloss'][i])
    cf_df['MaxEpochs'].loc[i] = nr_epochs_trained

cf_df

,HoV,c_f_hat,FlowRate_Diff_(LTR-FDDN),f1,f2,Final_MSELoss,Final_ErrorLoss,MaxEpochs
0,A1,1.024306,0.000072,2.495516e-05,-2.028590e-07,5.155005e-09,4.415687e-09,3
1,A2,1.061594,0.000045,4.583340e-07,1.689663e-08,1.983135e-09,1.698719e-09,1
2,A3,1.151122,0.000163,2.141496e-05,4.353169e-07,2.648402e-08,2.268575e-08,1
3,A4,1.300599,0.000312,5.018005e-06,-4.011776e-07,9.740244e-08,8.343323e-08,1
4,A5,0.999712,-0.000521,1.103821e-05,1.019896e-06,2.711626e-07,2.322731e-07,1
5,A6,0.933408,-0.001815,2.553274e-05,-5.343178e-06,3.293945e-06,2.821536e-06,1
6,C1,0.988615,0.000093,1.079061e-05,-1.779225e-07,8.706351e-09,7.457708e-09,1
7,C2,1.261821,-0.000157,8.762826e-07,8.587356e-08,2.472185e-08,2.117631e-08,1
8,C3,1.239185,-0.000114,1.685815e-06,8.683793e-08,1.304982e-08,1.117825e-08,1
9,C4,1.048186,-0.000280,8.917578e-06,-4.891083e-07,7.835356e-08,6.711629e-08,1


In [24]:
print('Max no. of epochs for:',cf_df['HoV'].iloc[[cf_df.MaxEpochs.idxmax()]].values)
cf_df.to_csv('data_output/NN_1CF_Output_FE_LS_SingleHoVLoop.csv',index=False)
cf_df.iloc[[cf_df.MaxEpochs.idxmax()]]

Max no. of epochs for: ['A1']


,HoV,c_f_hat,FlowRate_Diff_(LTR-FDDN),f1,f2,Final_MSELoss,Final_ErrorLoss,MaxEpochs
0,A1,1.024306,0.000072,0.000025,-2.028590e-07,5.155005e-09,4.415687e-09,3


### Plot Training Loss

In [25]:
from bokeh.palettes import Category20,Colorblind,Spectral,Set2,YlGnBu,RdPu
temp_list = []
bokeh_palettes = [Colorblind,YlGnBu,RdPu,Set2]
for palette in bokeh_palettes:
    for key in palette.keys():
        temp_list.append(palette[key])
    
color_palette = [y for x in temp_list for y in x]
print(color_palette)

['#0072B2', '#E69F00', '#F0E442', '#0072B2', '#E69F00', '#F0E442', '#009E73', '#0072B2', '#E69F00', '#F0E442', '#009E73', '#56B4E9', '#0072B2', '#E69F00', '#F0E442', '#009E73', '#56B4E9', '#D55E00', '#0072B2', '#E69F00', '#F0E442', '#009E73', '#56B4E9', '#D55E00', '#CC79A7', '#0072B2', '#E69F00', '#F0E442', '#009E73', '#56B4E9', '#D55E00', '#CC79A7', '#000000', '#2c7fb8', '#7fcdbb', '#edf8b1', '#225ea8', '#41b6c4', '#a1dab4', '#ffffcc', '#253494', '#2c7fb8', '#41b6c4', '#a1dab4', '#ffffcc', '#253494', '#2c7fb8', '#41b6c4', '#7fcdbb', '#c7e9b4', '#ffffcc', '#0c2c84', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4', '#ffffcc', '#0c2c84', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4', '#edf8b1', '#ffffd9', '#081d58', '#253494', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4', '#edf8b1', '#ffffd9', '#c51b8a', '#fa9fb5', '#fde0dd', '#ae017e', '#f768a1', '#fbb4b9', '#feebe2', '#7a0177', '#c51b8a', '#f768a1', '#fbb4b9', '#feebe2', '#7a0177', '#c51b8a', '#f768a1', '#fa9fb5'

In [26]:
num_lines = len(losses.HoV) # no. of lines to draw
colors = color_palette[0:num_lines]
labels = losses.HoV.values.tolist()

p1 = figure(width=1600)
for i in range(num_lines):
    x = list(range(1,len(losses['MSE_flowloss'][i])+1))
    y1 = losses['MSE_flowloss'][i]    
    p1.line(x, y1, line_width=2, color=colors[i], alpha=0.8, legend='MSE Flow Loss for {}'.format(labels[i]))    
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
p1.legend.click_policy="hide"
show(p1)

In [27]:
p2 = figure(width=1600)
for i in range(num_lines):
    x = list(range(1,len(losses['MSE_flowloss'][i])+1))     
    y2 = losses['Error_loss'][i]    
    err_loss_lineplot = p2.line(x, y2, line_width=2, color= colors[i], alpha=0.5, legend='Error Loss for {}'.format(labels[i]))
p2.yaxis.axis_label = "Error Loss"
p2.xaxis.axis_label = "Epochs"
p2.legend.click_policy="hide"
show(p2)